## Imports

In [1]:

import os
# os.chdir('../')
# print(os.getcwd())
if "with_dynamic_edge" in str(os.getcwd()):
    os.chdir('../')
"""
    IMPORTING LIBS
"""
import dgl

import numpy as np
import os
import socket
import time
import random
import glob
import argparse, json
import pickle

import torch
import torch.nn as nn
import torch.nn.functional as F

import torch.optim as optim
from torch.utils.data import DataLoader

from tensorboardX import SummaryWriter
from tqdm import tqdm

import random
import dgl
import torch
import torch.nn as nn
import torch.nn.functional as F
import itertools
import numpy as np
import scipy.sparse as sp

import pandas as pd

import dgl.function as fn
MODEL_NAME = 'GraphSage'

class DotDict(dict):
    def __init__(self, **kwds):
        self.update(kwds)
        self.__dict__ = self

"""
    IMPORTING CUSTOM MODULES/METHODS
"""
# from nets.COLLAB_edge_classification.load_net import gnn_model # import all GNNS
from nets.COLLAB_edge_classification.load_net import gnn_model
from data.data import LoadData
"""
    GPU Setup
"""
def gpu_setup(use_gpu, gpu_id):
    os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
    os.environ["CUDA_VISIBLE_DEVICES"] = str(gpu_id)  

    if torch.cuda.is_available() and use_gpu:
        print('cuda available with GPU:',torch.cuda.get_device_name(0))
        device = torch.device("cuda")
    else:
        print('cuda not available')
        device = torch.device("cpu")
    return device
use_gpu = False; gpu_id = -1; device = None # CPU
# """
#     USER CONTROLS
# """

/usr/local/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Load Data

In [2]:
def load_datset(DATASET_NAME):
    print("[I] Loading data (notebook) ...")
    dataset = LoadData(DATASET_NAME)
    print("[I] Finished loading.....")
    # print(dataset)

    return dataset

## DEFINE PARAMETER

In [3]:

def define_parameter(MODEL_NAME, dataset):
    
#     MODEL_NAME = 'MF'
    # MODEL_NAME = 'GatedGCN'
    
    n_heads = -1
    edge_feat = False
    pseudo_dim_MoNet = -1
    kernel = -1
    gnn_per_block = -1
    embedding_dim = -1
    pool_ratio = -1
    n_mlp_GIN = -1
    gated = False
    self_loop = False
    max_time = 12
    layer_type = 'dgl'
    num_embs = -1
    pos_enc = True
    #pos_enc = False
    pos_enc_dim = 10

    
    if MODEL_NAME == 'MF':
        seed=41; epochs=500; batch_size=32*1024; init_lr=0.01; lr_reduce_factor=0.5; lr_schedule_patience=10; min_lr = 1e-5; weight_decay=0
        L=0; hidden_dim=256; out_dim=hidden_dim; num_embs=235868;
    
    if MODEL_NAME == 'MLP':
        seed=41; epochs=500; batch_size=32*1024; init_lr=0.001; lr_reduce_factor=0.5; lr_schedule_patience=10; min_lr = 1e-5; weight_decay=0
        L=3; hidden_dim=80; out_dim=hidden_dim; dropout=0.0; readout='mean'; gated = False  # Change gated = True for Gated MLP model
    
    if MODEL_NAME == 'GCN':
        seed=41; epochs=500; batch_size=32*1024; init_lr=0.001; lr_reduce_factor=0.5; lr_schedule_patience=10; min_lr = 1e-5; weight_decay=0
        L=5; hidden_dim=74; out_dim=hidden_dim; dropout=0.0; readout='mean';
        
    if MODEL_NAME == 'GraphSage':
        seed=41; epochs=500; batch_size=32*1024; init_lr=0.001; lr_reduce_factor=0.5; lr_schedule_patience=10; min_lr = 1e-5; weight_decay=0
        L=10; hidden_dim=38; out_dim=hidden_dim; dropout=0.0; readout='mean'; layer_type='edgefeat'

    if MODEL_NAME == 'GAT':
        seed=41; epochs=500; batch_size=32*1024; init_lr=0.001; lr_reduce_factor=0.5; lr_schedule_patience=10; min_lr = 1e-5; weight_decay=0
        L=3; n_heads=3; hidden_dim=19; out_dim=n_heads*hidden_dim; dropout=0.0; readout='mean'; layer_type='dgl'
    
    if MODEL_NAME == 'GIN':
        seed=41; epochs=500; batch_size=32*1024; init_lr=0.001; lr_reduce_factor=0.5; lr_schedule_patience=10; min_lr = 1e-5; weight_decay=0
        L=3; hidden_dim=60; out_dim=hidden_dim; dropout=0.0; readout='mean';
        
    if MODEL_NAME == 'MoNet':
        seed=41; epochs=500; batch_size=32*1024; init_lr=0.001; lr_reduce_factor=0.5; lr_schedule_patience=10; min_lr = 1e-5; weight_decay=0
        L=3; hidden_dim=53; out_dim=hidden_dim; dropout=0.0; readout='mean';
        
    if MODEL_NAME == 'GatedGCN':
        seed=41; epochs=500; batch_size=32*1024; init_lr=0.001; lr_reduce_factor=0.5; lr_schedule_patience=10; min_lr = 1e-5; weight_decay=0
        L=5; hidden_dim=35; out_dim=hidden_dim; dropout=0.0; readout='mean'; edge_feat = False; layer_type='edgereprfeat'
        
    # generic new_params
    net_params = {}
    net_params['device'] = device
    net_params['in_dim'] = dataset.graph.ndata['feat'].shape[-1]
    net_params['in_dim_edge'] = dataset.graph.edata['feat'].shape[-1]
    net_params['residual'] = True
    net_params['hidden_dim'] = hidden_dim
    net_params['out_dim'] = out_dim
    num_classes = 1
    net_params['n_classes'] = num_classes
    net_params['n_heads'] = n_heads
    net_params['L'] = L  # min L should be 2
    net_params['readout'] = "mean"
    net_params['layer_norm'] = True
    net_params['batch_norm'] = True
    net_params['in_feat_dropout'] = 0.0
    net_params['dropout'] = 0.0
    net_params['edge_feat'] = edge_feat
    net_params['self_loop'] = self_loop
    net_params['layer_type'] = layer_type
    
    # for MF
    net_params['num_embs'] = num_embs
    
    # for MLPNet 
    net_params['gated'] = gated
    
    # specific for MoNet
    net_params['pseudo_dim_MoNet'] = 2
    net_params['kernel'] = 3
    
    # specific for GIN
    net_params['n_mlp_GIN'] = 2
    net_params['learn_eps_GIN'] = True
    net_params['neighbor_aggr_GIN'] = 'sum'
    
    # specific for graphsage
    net_params['sage_aggregator'] = 'maxpool'   
    
    # specific for pos_enc_dim
    net_params['pos_enc'] = pos_enc
    net_params['pos_enc_dim'] = pos_enc_dim

    
    params = {}
    params['seed'] = seed
    params['epochs'] = epochs
    params['batch_size'] = batch_size
    params['init_lr'] = init_lr
    params['lr_reduce_factor'] = lr_reduce_factor 
    params['lr_schedule_patience'] = lr_schedule_patience
    params['min_lr'] = min_lr
    params['weight_decay'] = weight_decay
    params['print_epoch_interval'] = 5
    params['max_time'] = max_time

    return net_params, params
    


## Load Data

In [4]:
def LoadDF(APPLICATION_NAME):
    df = pd.read_csv("../new/"+APPLICATION_NAME+"_node.csv")
    return df  

## DEFINE PARAMETER

In [5]:
def define_parameter(MODEL_NAME, dataset):
    
#     MODEL_NAME = 'MF'
    # MODEL_NAME = 'GatedGCN'
    
    n_heads = -1
    edge_feat = False
    pseudo_dim_MoNet = -1
    kernel = -1
    gnn_per_block = -1
    embedding_dim = -1
    pool_ratio = -1
    n_mlp_GIN = -1
    gated = False
    self_loop = False
    max_time = 12
    layer_type = 'dgl'
    num_embs = -1
    pos_enc = True
    #pos_enc = False
    pos_enc_dim = 10

    
    if MODEL_NAME == 'MF':
        seed=41; epochs=500; batch_size=32*1024; init_lr=0.01; lr_reduce_factor=0.5; lr_schedule_patience=10; min_lr = 1e-5; weight_decay=0
        L=0; hidden_dim=256; out_dim=hidden_dim; num_embs=235868;
    
    if MODEL_NAME == 'MLP':
        seed=41; epochs=500; batch_size=32*1024; init_lr=0.001; lr_reduce_factor=0.5; lr_schedule_patience=10; min_lr = 1e-5; weight_decay=0
        L=3; hidden_dim=80; out_dim=hidden_dim; dropout=0.0; readout='mean'; gated = False  # Change gated = True for Gated MLP model
    
    if MODEL_NAME == 'GCN':
        seed=41; epochs=500; batch_size=32*1024; init_lr=0.001; lr_reduce_factor=0.5; lr_schedule_patience=10; min_lr = 1e-5; weight_decay=0
        L=5; hidden_dim=74; out_dim=hidden_dim; dropout=0.0; readout='mean';
        
    if MODEL_NAME == 'GraphSage':
        seed=41; epochs=500; batch_size=32*1024; init_lr=0.001; lr_reduce_factor=0.5; lr_schedule_patience=10; min_lr = 1e-5; weight_decay=0
        L=10; hidden_dim=38; out_dim=hidden_dim; dropout=0.0; readout='mean'; layer_type='edgefeat'

    if MODEL_NAME == 'GAT':
        seed=41; epochs=500; batch_size=32*1024; init_lr=0.001; lr_reduce_factor=0.5; lr_schedule_patience=10; min_lr = 1e-5; weight_decay=0
        L=3; n_heads=3; hidden_dim=19; out_dim=n_heads*hidden_dim; dropout=0.0; readout='mean'; layer_type='dgl'
    
    if MODEL_NAME == 'GIN':
        seed=41; epochs=500; batch_size=32*1024; init_lr=0.001; lr_reduce_factor=0.5; lr_schedule_patience=10; min_lr = 1e-5; weight_decay=0
        L=3; hidden_dim=60; out_dim=hidden_dim; dropout=0.0; readout='mean';
        
    if MODEL_NAME == 'MoNet':
        seed=41; epochs=500; batch_size=32*1024; init_lr=0.001; lr_reduce_factor=0.5; lr_schedule_patience=10; min_lr = 1e-5; weight_decay=0
        L=3; hidden_dim=53; out_dim=hidden_dim; dropout=0.0; readout='mean';
        
    if MODEL_NAME == 'GatedGCN':
        seed=41; epochs=500; batch_size=32*1024; init_lr=0.001; lr_reduce_factor=0.5; lr_schedule_patience=10; min_lr = 1e-5; weight_decay=0
        L=5; hidden_dim=35; out_dim=hidden_dim; dropout=0.0; readout='mean'; edge_feat = False; layer_type='edgereprfeat'
        
    # generic new_params
    net_params = {}
    net_params['device'] = device
    net_params['in_dim'] = dataset.graph.ndata['feat'].shape[-1]
    net_params['in_dim_edge'] = dataset.graph.edata['feat'].shape[-1]
    net_params['residual'] = True
    net_params['hidden_dim'] = hidden_dim
    net_params['out_dim'] = out_dim
    num_classes = 1
    net_params['n_classes'] = num_classes
    net_params['n_heads'] = n_heads
    net_params['L'] = L  # min L should be 2
    net_params['readout'] = "mean"
    net_params['layer_norm'] = True
    net_params['batch_norm'] = True
    net_params['in_feat_dropout'] = 0.0
    net_params['dropout'] = 0.0
    net_params['edge_feat'] = edge_feat
    net_params['self_loop'] = self_loop
    net_params['layer_type'] = layer_type
    
    # for MF
    net_params['num_embs'] = num_embs
    
    # for MLPNet 
    net_params['gated'] = gated
    
    # specific for MoNet
    net_params['pseudo_dim_MoNet'] = 2
    net_params['kernel'] = 3
    
    # specific for GIN
    net_params['n_mlp_GIN'] = 2
    net_params['learn_eps_GIN'] = True
    net_params['neighbor_aggr_GIN'] = 'sum'
    
    # specific for graphsage
    net_params['sage_aggregator'] = 'maxpool'   
    
    # specific for pos_enc_dim
    net_params['pos_enc'] = pos_enc
    net_params['pos_enc_dim'] = pos_enc_dim

    
    params = {}
    params['seed'] = seed
    params['epochs'] = epochs
    params['batch_size'] = batch_size
    params['init_lr'] = init_lr
    params['lr_reduce_factor'] = lr_reduce_factor 
    params['lr_schedule_patience'] = lr_schedule_patience
    params['min_lr'] = min_lr
    params['weight_decay'] = weight_decay
    params['print_epoch_interval'] = 5
    params['max_time'] = max_time

    return net_params, params
    


## VIEW MODEL PARAMETER

In [6]:

def view_model_param(MODEL_NAME, net_params):
    # print(net_params)
    model = gnn_model(MODEL_NAME, net_params)
    total_param = 0
    # print("MODEL DETAILS:\n")
    # print(model)
    for param in model.parameters():
        # print(param.data.size())
        total_param += np.prod(list(param.data.size()))
    # print('MODEL/Total parameters:', MODEL_NAME, total_param)
    return total_param

tim1 = time.time()


## Set Parameters

In [7]:
def set_parameters(MODEL_NAME, dataset, DATASET_NAME):
    net_params, params = define_parameter(MODEL_NAME=MODEL_NAME, dataset=dataset)
    config = {}
    gpu = {}
    gpu['use'] = use_gpu
    gpu['id'] = gpu_id
    config['gpu'] = gpu
    # GNN model, dataset, out_dir
    config['model'] = MODEL_NAME
    config['dataset'] = DATASET_NAME
    out_dir = 'out/debug/'
    config['out_dir'] = out_dir
    config['params'] = params
    # network parameters
    config['net_params'] = net_params
    params = config['params']
    DATASET_NAME = config['dataset']
    device = gpu_setup(config['gpu']['use'], config['gpu']['id'])
    out_dir = config['out_dir']
    MODEL_NAME = config['model']
    net_params = config['net_params']
    net_params['device'] = device
    net_params['gpu_id'] = config['gpu']['id']
    net_params['batch_size'] = params['batch_size']
    net_params['in_dim'] = dataset.graph.ndata['feat'].shape[-1]
    net_params['in_dim_edge'] = dataset.graph.edata['feat'].shape[-1]
    net_params['n_classes'] = 1  # binary prediction
    net_params['total_param'] = view_model_param(MODEL_NAME, net_params)
    
    return net_params, params

## Get Epoc Number

In [8]:
def get_EPOC_Number(PATH):
    file_path = PATH+"/ROC_CURVE/*"
    project_list = glob.glob(file_path)
    max_ = -1
    for line in project_list:
        index = line.rfind("_")
        last_index = line.rfind(".png")
        number = int(line[index+1:last_index])
        max_ = max(max_, number)
    # print(max_)
    return max_

## Train Model

In [9]:
def train_model(dataset, EPOC_NUMBER, APPLICATION_NAME, net_params, params, MODEL_NAME):
    t0 = time.time()    
    DATASET_NAME = dataset.name

    if MODEL_NAME in ['GatedGCN']:
        if net_params['pos_enc']:
            print("[!] Adding graph positional encoding",net_params['pos_enc_dim'])
            dataset._add_positional_encodings(net_params['pos_enc_dim'])
            print('Time PE:',time.time()-t0)

    graph = dataset.graph
    evaluator=""
    train_edges, val_edges, val_edges_neg, test_edges, test_edges_neg = dataset.train_edges, dataset.val_edges, dataset.val_edges_neg, dataset.test_edges, dataset.test_edges_neg
    device = net_params['device']
    random.seed(params['seed'])
    np.random.seed(params['seed'])
    torch.manual_seed(params['seed'])
    if device.type == 'cuda':
        torch.cuda.manual_seed(params['seed'])

    print("Graph: ", graph)
    print("Training Edges: ", len(train_edges))
    print("Validation Edges: ", len(val_edges) + len(val_edges_neg))
    print("Test Edges: ", len(test_edges) + len(test_edges_neg))

    print(net_params)
    model = gnn_model(MODEL_NAME, net_params)
    model = model.to(device)
    out_dir = 'out/debug/'
    PATH = out_dir + 'checkpoints/' + MODEL_NAME + "_NEW_DATASET--" + APPLICATION_NAME + "/RUN_/epoch_"+str(EPOC_NUMBER)+".pkl"
    t = torch.load(PATH)
    model.load_state_dict(torch.load(PATH))
#     print(model)

    optimizer = optim.Adam(model.parameters(), lr=params['init_lr'], weight_decay=params['weight_decay'])
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='max',
                                                        factor=params['lr_reduce_factor'],
                                                        patience=params['lr_schedule_patience'],
                                                        verbose=True)

    return model, graph

## Get Test Data

In [10]:
def get_test_data(test_size, APPLICATION_NAME=""):
    ROOT_PATH = "../new/"
    df = pd.read_csv(ROOT_PATH+ APPLICATION_NAME+'_node.csv')
    nodes_data = df.drop(["start_line","start_column","end_line",  "end_column" ,"file_name"], axis=1)
    
    stmt_type=['FunctionDeclaration', 'ArrowFunctionExpression', 'FunctionExpression']
    df1 = nodes_data[nodes_data.type.isin(stmt_type)]
    test_neg_id = df1['new_id'].tolist()

    df1 = nodes_data[((nodes_data['type']=='CallExpression')|(nodes_data['type']=='NewExpression'))]
    test_neg_call_site = df1['new_id'].tolist()

    test_edges= pd.read_csv(ROOT_PATH+ APPLICATION_NAME +'_function_edges.csv')

    test_edges.columns = ["src", "dst"]
    index = pd.MultiIndex.from_product([test_neg_call_site, test_neg_id], names = ["src", "dst"])
    all_combi = pd.DataFrame(index = index).reset_index()
    
    test_neg_df =  pd.merge(all_combi,test_edges, indicator=True, how='outer').query('_merge=="left_only"').drop('_merge', axis=1)
    test_neg_df = test_neg_df.iloc[np.random.permutation(len(test_neg_df))]
    # print(test_neg_df)
    test_neg_df = test_neg_df[:test_size]
    test_edges = torch.from_numpy(test_neg_df.to_numpy())
    return test_edges
#     return test_neg_df

## Get Cutoff Threshold

In [11]:
def get_thresholds(PATH, EPOC_NUMBER):
    all_list =[[],[],[]]
    k=0
    with open(PATH+"/THRESHOLD/threshold_values_with_"+str(EPOC_NUMBER)+".txt") as in_file:
        for line in in_file:
            parts = line.split(" ")
            # print(parts)
            for part in parts:
                try:
                    numeric_string = float(''.join(c for c in part if (c.isdigit() or c =='.' or c=='e' or c=='-')))
                    # print(numeric_string)
                    all_list[k].append(numeric_string)
                except:
                    pass
            if "]" in line:
                k+=1

    threshold_lst = all_list[0]
    fpr_lst = all_list[1]
    tpr_lst = all_list[2]

    # print(threshold_lst)
    for i in range(len(fpr_lst)):
        if fpr_lst[i]!=0.0:
            first_index = i-1
            break
    

    first_thrshld = threshold_lst[first_index]

    return first_thrshld


In [12]:
# APPLICATION_NAME = 'express'
# PATH = "results/NEW_DATASET--"+APPLICATION_NAME+"/"+MODEL_NAME
# EPOC_NUMBER = get_EPOC_Number(PATH=PATH)
# print(EPOC_NUMBER)
# print(get_thresholds(PATH=PATH, EPOC_NUMBER=EPOC_NUMBER))


## Evaluate Network

In [13]:
def evaluate_network(model, device, graph, test_edges,
                     batch_size, DATASET_NAME="", MODEL_NAME="", SOURCE_NODE=0, DST_NODE=0, APPLICATION_NAME="", EPOC_NUMBER=0):
    
    model.eval()
    with torch.no_grad():
        graph = graph.to(device)
        x = graph.ndata['feat'].to(device)
        e = graph.edata['feat'].to(device).float()
        try:
            x_pos_enc = graph.ndata['pos_enc'].to(device)
            h = model(graph, x, e, x_pos_enc) 
        except:
            h = model(graph, x, e)

        test_edges = test_edges.to(device)

        test_preds = []
        for perm in DataLoader(range(test_edges.size(0)), batch_size):
            edge = test_edges[perm].t()
            # print("edge ==> ",edge, len(edge))
            test_preds += [model.edge_predictor(h[edge[0]], h[edge[1]]).squeeze().cpu()]
        # print(test_preds)
        if len(test_edges)==1:
            t= test_preds[0]
            t =np.expand_dims(t,0)
            t = torch.tensor(t)
            test_preds = [t]
        test_pred = torch.cat(test_preds, dim=0)
        print(test_pred)
        
    cutoff_thrshld = get_thresholds(PATH=PATH, EPOC_NUMBER=EPOC_NUMBER)
    cutoff_thrshld = "{:.12f}".format(float(cutoff_thrshld))
    count = 0
    for i in range(len(test_pred)):
        pred_score = "{:.12f}".format(float(test_pred[i].item()))
        if(pred_score >= cutoff_thrshld):
           count+=1
    
    print(count)
        
#     write_to_file(test_edges=test_edges, test_pred=test_pred, src_id=SOURCE_NODE, APPLICATION_NAME=APPLICATION_NAME)

#     src=[]
#     dst=[]
#     scores = []
#     for i in range(len(test_edges)):
#         pred_score = "{:.12f}".format(float(test_pred[i].item()))
#         s = int(test_edges[i][0].item())
#         d = int(test_edges[i][1].item())
#         src.append(s)
#         dst.append(d)
#         scores.append(pred_score)

#     d = {"src": src, "dst":dst, "score":scores}
#     df = pd.DataFrame(d)
#     df = df.sort_values(by='score', ascending=False, na_position='first').reset_index(drop=True)
#     temp_df = df[(df["src"]==SOURCE_NODE) & (df["dst"]== DST_NODE)]
#     if len(temp_df)!=0:
#         print("ACTUAL EDGE RANK =====> ", temp_df.index[0])
#         return temp_df.index[0]
#     else:
#         print(SOURCE_NODE, DST_NODE)
#         print(df)
#     return -1

## Get Candidate

In [14]:
def get_candidate(PATH, EPOC_NUMBER, model, graph, params, DATASET_NAME, MODEL_NAME, APPLICATION_NAME):
    
#     for i in range(len(pos_df)):
#         src_id = int(pos_df.iloc[i][0])
#         dst_id = int(pos_df.iloc[i][1])
# #         print(src_id, dst_id)
#         try:
#             start = time.time()
#             test_edges = get_test_data(test_size=10000, APPLICATION_NAME=APPLICATION_NAME)
# #             # print(len(test_edges), src_id, dst_id)
# #             if len(test_edges) > 0:
# #                 rank = evaluate_network(
# #                         model, device, graph, test_edges, params['batch_size'], DATASET_NAME=DATASET_NAME, MODEL_NAME=MODEL_NAME, SOURCE_NODE=src_id, DST_NODE=dst_id, APPLICATION_NAME=APPLICATION_NAME)
# #             # if src_name == dst_name:
# #                 if rank!= -1:
# #                     if rank<=5:
# #                     # print(rank)
# #                         count+=1
# #                     else:
# #                         not_top+=1
# #                 else:
# #                     print("PROBLEM!!!")
# #                 rank_list.append(rank)
#         except KeyboardInterrupt:
#             print('-' * 89)
#             print('Exiting from training early because of KeyboardInterrupt')

# #                 # break

# #     return rank_list


    test_edges = get_test_data(test_size=10000, APPLICATION_NAME=APPLICATION_NAME)
    evaluate_network(model, device, graph, test_edges, params['batch_size'], DATASET_NAME=DATASET_NAME, MODEL_NAME=MODEL_NAME, APPLICATION_NAME=APPLICATION_NAME, EPOC_NUMBER=EPOC_NUMBER)
    

## Main Function

In [ ]:
MODEL_NAME = 'GatedGCN'
# to_do_list = ['js-data', 'atropa-ide', 'mathjs','mixin-pro', 'locutus', 'is-my-json-valid', 'ms' ]
# app_list = ['js-data_3.0.9', 'atropa-ide_0.2.2-2', 'mathjs_3.9.0','mixin-pro_0.6.0', 'locutus_2.0.10', 'is-my-json-valid_2.20.0', 'ms_0.7.0']

# to_do_list = ['lodash', 'formula-parser', 'mathjs' ]
# app_list = ['lodash', 'formula-parser', 'mathjs' ]

to_do_list = ['formula-parser', 'lodash', 'express','js-yaml']
app_list =  ['formula-parser', 'lodash', 'express','js-yaml']

# to_do_list = ['is-my-json-valid']
# app_list = ['is-my-json-valid_2.20.0']

print(os.getcwd())
if "with_dynamic_edge" not in str(os.getcwd()):
    os.chdir('with_dynamic_edge/')
print(os.getcwd())
for i in range(len(to_do_list)):
    APPLICATION_NAME = app_list[i]
    DATASET_NAME = 'NEW_DATASET--'+APPLICATION_NAME
    print("\033[97m ------------------------------------------------------------------")
    print("\033[97m ", APPLICATION_NAME)
    print("\033[97m ------------------------------------------------------------------")
    # print("[I] Loading data (notebook) ...")
    node_df = LoadDF(APPLICATION_NAME)
    # print(node_df)
    # print("[I] Finished loading.")
    PATH = "results/NEW_DATASET--"+APPLICATION_NAME+"/"+MODEL_NAME
    EPOC_NUMBER = get_EPOC_Number(PATH=PATH)
    print(EPOC_NUMBER)
    dataset = load_datset(DATASET_NAME)
    net_params, params = set_parameters(MODEL_NAME, dataset, DATASET_NAME)
    model, graph = train_model(dataset, EPOC_NUMBER, APPLICATION_NAME, net_params, params, MODEL_NAME)
    # print(model, graph)
    rank_list = get_candidate(PATH, EPOC_NUMBER, model, graph, params, DATASET_NAME, MODEL_NAME, APPLICATION_NAME)
#     plot_graph(rank_list, APPLICATION_NAME)

In [24]:
import glob
import pandas as pd
folders = glob.glob("../prune_new/*")
project_list = ['moment', 'jsPDF', 'webpack', 'atompm', 'jquery', 'ws', 'cheerio', 'yargs', 'eslint-plugin-react', 'socket.io', 'sass-loader', 'handlebars.js']
count = 0
lst=[]
# for file_ in folders:
#     first_index = file_.rfind("/")
#     last_index = file_.rfind("_function_edges.csv")
#     DATASET_NAME = file_[first_index+1:last_index]
#     if "function_edges.csv" in file_:
#         df =pd.read_csv(file_)
#         if DATASET_NAME in project_list and len(df)>500:
#             print(DATASET_NAME,"=======>",len(df))
#             lst.append(DATASET_NAME)
#             count+=1

# print(lst)

# for file_ in folders:
#     first_index = file_.rfind("/")
#     last_index = file_.rfind("_node")
#     DATASET_NAME = file_[first_index+1:last_index]
#     if "_node" in file_:
#         df =pd.read_csv(file_)
#         if DATASET_NAME in project_list and len(df)>500:
#             print(DATASET_NAME,"=======>",len(df))
#             lst.append(DATASET_NAME)
#             count+=1

# print(lst)
from os.path import exists
count = 0
lst = []
folders = glob.glob("results/*")
for folder_path in folders:
    first_index = folder_path.rfind("--")
    DATASET_NAME = folder_path[first_index+2:]
    file_exists = exists(folder_path+"/GatedGCN/train_loss.png")
    if file_exists:
        df = pd.read_csv(folder_path+"/GatedGCN/POS_PRED/pred_score_with_0_pos.csv", header=None)
        if len(df)>50:
            # print(DATASET_NAME, "======> ",len(df))
            lst.append(DATASET_NAME) 
# print(count)
print(lst)

['lodash', 'underscore', 'eslint-plugin-import', 'mocha', 'angular', 'ramda', 'shelljs', 'js-yaml', 'commander.js', 'bluebird', 'express', 'async', 'jsPDF', 'jquery', 'coffeescript', 'handlebars.js', 'bootstrap', 'immutable-js', 'package', 'q', 'formula-parser', 'atompm', 'request', 'mongoose', 'mathjs', 'less.js', 'webpack', 'eslint-plugin-react', 'axios', 'react']
